# Transmisión de señales por un canal de comunicaciones
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para explicar los fundamentos de los <code>Sistemas de comunicaciones</code> en los cursos de telemática y redes de computadores.</p>

## Descripción
Este algoritmo, también llamado de caminos mínimos, es un método para resolver el problema del camino mas corto.

### Agenda
1. Factores que afectan la transmisión.
1. Tasas de transferencia.
1. Capacidad del canal.

In [2]:
# Importar módulos al cuaderno
import math as m
import matplotlib.pyplot as plt

# Importar elementos de un módulo al cuaderno
from math import pi
from numpy import sin

## 1. Factores que afectan la transmisión  ✔

### 1.1. La atenuación 
la [Atenuación][alpha] es perdida de potencia de la señal durante su propagación por el medio, depende de las características del medio y de la frecuencia.


[alpha]:https://es.wikipedia.org/wiki/Atenuación

#### 1.1.1. Causas:
* Absorción del material.
* Oposición Transmisión.
* Defectos de fabricación.
* Deformación del material.

#### 1.1.2. Representación:

![Representación](../img/attenuation_utp.svg "Atenuación")

#### 1.1.3. Ecuaciones:
$$
\begin{aligned}
\alpha  &= 10 \cdot log_{10}\left(\frac{P_o}{P_i}\right) \\ 
        &= 20 \cdot log_{10}\left(\frac{V_o}{V_i}\right) \\
\end{aligned}
$$

In [18]:
def voltage_attenuation(voltage_out:float, voltage_in:float):
    ''' Calcula la atenuación desde los valores de voltaje de la señal de entrada. '''
    return round(20 * m.log10(voltage_out/voltage_in))

def voltage_out(decibels:float, voltage_in:float=100):
    ''' Calcula el voltaje de salida desde los valores de atenuación de la señal de entrada. '''
    return round(voltage_in * (10 ** (decibels/20)), 2)

def power_attenuation(power_out:float, power_in:float):
    ''' Calcula la atenuación desde los valores de potencia de la señal de entrada. '''
    return round(10 * m.log10(power_out/power_in))

def power_out(decibels:float, power_in:float=100):
    ''' Calcula la potencia de salida desde los valores de atenuación de la señal de entrada. '''
    return round(power_in * (10 ** (decibels/10)), 2)


print(f"Atenuación de voltaje:  {voltage_attenuation(70.7, 100)} dB")
print(f"Voltaje de salida: {voltage_out(-30)} voltios")
print(20*"--")
print(f"Atenuación de potencia: {power_attenuation(50, 100)} dB")
print(f"Voltaje de salida: {power_out(-30)} vatios")


Atenuación de voltaje:  -3 dB
Voltaje de salida: 3.16 voltios
----------------------------------------
Atenuación de potencia: -3 dB
Voltaje de salida: 0.1 vatios


### 1.2. El ruido 
El [Ruido][noise] son las potencias parásitas que afectan la transmisión.

[noise]:https://es.wikipedia.org/wiki/Ruido_(comunicación)#



In [ ]:
### 1.3. El eco

### 1.3. El eco
El [Eco][21_3] es reflexión de una parte de la potencia original.

[21_3]:https://es.wikipedia.org/wiki/P%C3%A9rdida_de_retorno